In [57]:
import os
import torchvision.models as models 
import torch.nn as nn
import torch
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torchvision
from torchinfo import summary
import numpy as np
import torch.nn.functional as F
import torchvision.transforms as T
from tqdm.auto import tqdm
import torchmetrics
import cv2
from glob import glob
import requests
from io import StringIO
from urllib.parse import urlparse
from tsai.all import *
os.environ["cuda_visible_devices"] = '0'
device0 = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size=1

In [58]:
url = "https://storage.googleapis.com/download/storage/v1/b/vitalring-seoul.appspot.com/o/ppgContinuous%2FsyFotyWOdfPVPfa8ZOkhJBodgpJ3%2FsyFotyWOdfPVPfa8ZOkhJBodgpJ3_1706757218000.csv?generation=1706757229270812&alt=media"
destination_path = "downloaded_file.csv"
response = requests.get(url)
response

<Response [200]>

In [59]:
if response.status_code == 200:
    # Assuming the CSV content is UTF-8 encoded
    csv_data = response.content.decode('utf-8')
    parsed_url = urlparse(url)
    filename = parsed_url.path.split('/')[-1]
    # Use pandas to read the CSV data into a DataFrame
    df = pd.read_csv(StringIO(csv_data))
    df.rename(columns={'PPG': 'AF', 'Type_1': 'PPG'}, inplace=True)
    # Now 'df' is a pandas DataFrame containing the CSV data
    print(df.head())
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

   AF       PPG
0   0  0.478063
1   0  0.458766
2   0  0.435854
3   0  0.419918
4   0  0.401380


In [60]:
import signal

class CustomDataset(Dataset):
    def __init__(self,data ,transform=None, target_transform=None):
        self.PPG_data=data
    def __len__(self):
        return len(self.PPG_data)

    def __getitem__(self, idx):

        signal_tensor=self.PPG_data[idx]
        
        return signal_tensor


test_data=torch.empty(len(df)//2500+1,1,3000)
for i in range(len(df)//2500):
    if len(df)//2500-1==i:
        signal_csv=df.loc[i*2500-499:(i+1)*2500]['PPG']
        test_data[i]=torch.tensor(signal_csv.to_numpy())
        signal_csv=df.loc[len(df)-3000:]['PPG']
        test_data[i+1]=torch.tensor(signal_csv.to_numpy())
    else:
        signal_csv=df.loc[i*2500:(i+1)*2500+499]['PPG']
        test_data[i]=torch.tensor(signal_csv.to_numpy())
        
test_dataset=CustomDataset(test_data)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [63]:
def test(model,dataloader):
    validation_tqdm=tqdm(dataloader)
    model.eval()
    val_count=0
    predict_list=[]
    with torch.no_grad():
        for x in validation_tqdm:
            val_count+=1
            x=x.to(device0).float()
            predict = model(x).to(device0)
            predict_list.append(F.sigmoid(predict).item())
    return predict_list

model =GRU_FCN(1,1,3000).to(device0)
model.load_state_dict(torch.load('../../model/architecture_waveform/PPG_GRU_FCN_callback.pt'), strict=False)
predict_list=test(model,test_dataloader)
predict_list

100%|██████████| 5/5 [00:00<00:00, 1148.43it/s]


[1.5435649402206764e-05,
 0.008535649627447128,
 0.0025514778681099415,
 0.0053351446986198425,
 0.006695276126265526]

In [69]:
np.array(predict_list).round(3)
str11='./data/ppgContinuous_syFotyWOdfPVPfa8ZOkhJBodgpJ3_syFotyWOdfPVPfa8ZOkhJBodgpJ3_1706756902000.csv'
filename = str11.split('/')[-1]
filename

'ppgContinuous_syFotyWOdfPVPfa8ZOkhJBodgpJ3_syFotyWOdfPVPfa8ZOkhJBodgpJ3_1706756902000.csv'

In [ ]:
import csv

def modify_csv(input_file, output_file):
    # CSV 파일 읽기
    with open(input_file, 'r', newline='') as infile:
        reader = csv.reader(infile)
        data = list(reader)

    # 데이터 수정 (예: 첫 번째 행의 첫 번째 열 값을 변경)
    data[0][0] = 'New Value'

    # 수정된 데이터를 CSV 파일에 쓰기
    with open(output_file, 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(data)